In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import itertools
import sklearn.metrics as met
import scipy as sp

import sys
sys.path.insert(0, '../../notebooks/libs/')
import FeatureExtraction as FE

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [2]:
training = pd.read_csv('../../csv/chat_based_features_training.csv')
test = pd.read_csv('../../csv/chat_based_features_test.csv')

training_sparse = sp.sparse.csr_matrix(training.ix[:,1:].values, dtype=float)[:,:-1]
test_sparse = sp.sparse.csr_matrix(test.ix[:,1:].values, dtype=float)[:,:-1]

In [ ]:
training_xml = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_xml = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

#matrix_training, matrix_testing = FE.make_tf_idf_matrix_from_XML(training_xml, test_xml)
documents_training = FE.prepare_for_tf_idf(training_xml, False)
documents_test = FE.prepare_for_tf_idf(test_xml, False)

In [ ]:
tfidf=TfidfVectorizer(stop_words='english', min_df=3, max_features=5000)
matrix_training=tfidf.fit_transform(documents_training)
matrix_testing=tfidf.transform(documents_test)

In [3]:
training_all = training_sparse
test_all = test_sparse
#training_all = sp.sparse.hstack((training_sparse, matrix_training))
#test_all = sp.sparse.hstack((test_sparse, matrix_testing))

In [ ]:
scaler=StandardScaler(with_mean=False)
training_all=scaler.fit_transform(training_all)
test_all=scaler.transform(test_all)

In [ ]:
column_names = training.columns.values.tolist()[1:-1]
print column_names

In [ ]:
#training.tail()

In [4]:
#forest = RandomForestClassifier(max_depth=17, n_estimators=300, bootstrap=False, n_jobs=8)
forest = RandomForestClassifier(n_estimators=150, n_jobs=8, bootstrap=False)

In [5]:
forest = forest.fit(training_all, training[['is sexual predator']])

/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


In [6]:
prediction = forest.predict(test_all)

In [7]:
print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
print 'F1:', met.f1_score(test[['is sexual predator']], prediction)

Accuracy:  0.99905807903
Precision:  0.730769230769
Recall: 0.299212598425
F1: 0.424581005587


In [ ]:
forest = RandomForestClassifier(n_estimators = 100)

max_f1 = 0
best_features = []

for num_features in range(2, len(column_names)-1):
    for column_name_subset in itertools.combinations(column_names, num_features):

        forest = forest.fit(training[list(column_name_subset)], np.ravel(training.iloc[:,[9]]))
        prediction = forest.predict(test[list(column_name_subset)])

        print column_name_subset
        print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
        print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
        print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
        print 'F1:', met.f1_score(test[['is sexual predator']], prediction)
        print "\n\n"

        f1 = met.f1_score(test[['is sexual predator']], prediction)
        if max_f1 < f1:
            max_f1 = f1
            best_features =  column_name_subset
                                                                                                 
print max_f1
print best_features